#### 토스 커뮤니티는 동적으로 생성되므로, selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [2]:
import datetime
from dateutil.relativedelta import relativedelta

In [3]:
import time

In [4]:
import FinanceDataReader as fdr

In [5]:
from data_loader import load_sector_data

In [5]:
kospi = fdr.StockListing('KOSPI')
kospi.head(1)

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,56200,1,100,0.18,56100,56500,55900,2194654,123260889300,335501779310000,5969782550,STK


In [6]:
kospi_list = kospi[['Code','Name']].head()

In [7]:
code = kospi_list[kospi_list['Name'] == '삼성전자']['Code'].values[0]

In [8]:
code

'005930'

In [6]:
# 종목명으로 업종코드 찾기

def search_code_by_name(market, name) :
  market_list = ['KOSPI','KOSDAQ','ETF']
  if market in market_list:
    if market in ['KOSPI','KOSDAQ']:
      df = fdr.StockListing(market)[['Code','Name']]
    else:
      df = fdr.StockListing('ETF/KR')[['Symbol','Name']]
  else:
    raise ValueError('시장입력오류')
  
  code = df[df['Name'] == name]['Code'].values[0]
  
  return code   
  

In [39]:
def code_list_by_market(market) :
  market_list = ['KOSPI','KOSDAQ','ETF']
  if market in market_list:
    if market in ['KOSPI','KOSDAQ']:
      df = fdr.StockListing(market).sort_values(by='Marcap', ascending=False)
      df = df.iloc[100:400]
    else:
      df = fdr.StockListing('ETF/KR').sort_values(by='MarCap', ascending=False).head(100)
      df.rename(columns={'Symbol': 'Code'}, inplace=True)
  else:
    raise ValueError('시장입력오류')
  
  code_list = df['Code'].tolist()
  return code_list

In [8]:
fdr.StockListing('KOSPI').sort_values(by='Marcap', ascending=False).head(100)

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,55300,2,-800,-1.43,56100,56500,55200,16059223,893461579301,330128975015000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,203500,2,-1500,-0.73,208000,212500,202500,6149475,1275141940365,148148481277500,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,,348500,2,-10000,-2.79,356000,357500,348000,280639,98184650500,81549000000000,234000000,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,1000000,2,-6000,-0.60,1001000,1020000,999000,42326,42628941000,71174000000000,71174000,STK
4,005380,KR7005380001,현대차,KOSPI,,226000,1,13000,6.10,215000,227000,215000,1951806,436680248000,47328059166000,209416191,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,035250,KR7035250000,강원랜드,KOSPI,,16750,1,230,1.39,16450,16800,16400,644153,10731556340,3583503375000,213940500,STK
96,028050,KR7028050003,삼성E&A,KOSPI,,17870,1,200,1.13,17510,18130,17500,991721,17621431000,3502520000000,196000000,STK
97,128940,KR7128940004,한미약품,KOSPI,,271000,2,-13000,-4.58,282000,282000,269500,96733,26478075500,3471778561000,12810991,STK
98,138930,KR7138930003,BNK금융지주,KOSPI,,10560,1,160,1.54,10380,10630,10340,827910,8701724580,3383811837120,320436727,STK


In [84]:
code_list = code_list_by_market('KOSPI')
code_list

['005930', '000660', '373220', '207940', '005380']

In [10]:
samsung = search_code_by_name('KOSPI', '삼성전자')
samsung

'005930'

In [11]:
url = f'https://tossinvest.com/stocks/A005930/community'

In [51]:
# 웹드라이버 설정
driver = webdriver.Chrome()  # 또는 다른 웹드라이버 사용
driver.get(url)

In [13]:
# 스크롤을 내리며 데이터 로드
last_height = driver.execute_script("return document.body.scrollHeight")

In [19]:
# 오늘 날짜 구하기
today = datetime.datetime.today()
today_str = today.strftime('%Y-%m-%d')

In [ ]:
code_list = code_list_by_market('KOSPI')
code_list

In [9]:
# 종목명으로 종목 코드를 찾는 함수
def search_code_by_name(name):
  kospi_df = fdr.StockListing('KOSPI')
  kosdaq_df = fdr.StockListing('KOSDAQ')
  
  all_stocks = pd.concat([kospi_df,kosdaq_df])
  
  code = all_stocks[all_stocks['Name'] == name]['Code'].values[0]
  
  return code

In [79]:
tmp = search_code_by_name('삼성전자')
tmp

'005930'

In [31]:
def crawling_community(market=None, start_date=None, end_date=today_str):
    
    if start_date is None :
        raise ValueError('시작일은 반드시 입력해야 합니다.')
    
    code_list = code_list_by_market(market)
    all_post_data = []
    
    for code in code_list:
        print(f"Processing {code} crawling...")
        url = f'https://tossinvest.com/stocks/A{code}/community'
        # 웹드라이버 초기화
        print(url)
        
        options = Options()
        options.add_argument('--headless')  # 헤드리스 모드
        driver = webdriver.Chrome(options=options)
        
        try:
            driver.get(url)  # 실제 URL로 변경
            WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '#stock-content > div > div > section > section > ul > div > div'))
                )
        except Exception as e:
            print(f"URL 접근 중 오류 발생: {e}. 종목 코드 {code}를 건너뜁니다.")
            driver.quit()  # 드라이버 종료
            continue  # 다음 종목 코드로 건너뜀
        
        # 버튼 요소 찾기
        button = driver.find_element(By.CSS_SELECTOR, '#stock-content > div > div > section > section > section > button')

        # data-contents-label 속성 값 확인
        label = button.get_attribute('data-contents-label')

        # '인기순'이라면 버튼 클릭하여 '최신순'으로 변경
        if label != '최신순':
            button.click()
            time.sleep(2)  # 변경 후 로딩 시간 대기
            
        # 게시글 리스트를 저장할 리스트
        all_contents = []
        post_nickname = []
        post_date = []

        # 날짜 기준 설정
        start_date = pd.to_datetime(start_date).date()  # tz-naive로 설정
        end_date = pd.to_datetime(end_date).date()

        # 스크롤 반복을 위한 변수
        new_posts_found = True
        processed_indices = set()

        while new_posts_found:    
            # 게시글 리스트 추출
            all_posts = driver.find_elements(By.CSS_SELECTOR, '#stock-content > div > div > section > section > ul > div > div')  # 실제 클래스명에 맞게 수정

            # article 태그가 포함된 div만 필터링
            filtered_posts = [post for post in all_posts if post.find_elements(By.TAG_NAME, 'article')]

            new_posts_found = False  # 새로운 게시글 발견 여부 초기화
            
            for post in filtered_posts:
                # data-index 추출
                index = post.get_attribute('data-index')
                
                # print(index)
                # 이미 처리한 인덱스인지 확인
                if index in processed_indices:
                    continue  # 이미 처리한 게시글은 건너뜀
                
                content = post.find_element(By.CSS_SELECTOR, 'article > div > a > span:nth-of-type(2) > span ')
                datetime = post.find_element(By.CSS_SELECTOR, 'article > div > header > div > label > time')
                date = datetime.get_attribute('datetime')
                nickname = post.find_element(By.CSS_SELECTOR, 'article > div > header > div > label > span:nth-child(1)')
                
                # print(content.text)
                # print(nickname.text)
                # 날짜 변환 (tz-naive)
                date_obj = pd.to_datetime(date).tz_localize(None).date()  # 날짜만 추출
                
                # 날짜 필터링
                if start_date <= date_obj <= end_date:
                    all_contents.append(content.text)
                    post_nickname.append(nickname.text)
                    post_date.append(date)
                    processed_indices.add(index)  # 인덱스를 집합에 추가
                    new_posts_found = True  # 새로운 게시글이 발견됨

            # 스크롤을 내림
            driver.execute_script("window.scrollBy(0, 3000);")
            time.sleep(2)  # 로딩 시간 대기

            # 특정 요소가 로드될 때까지 대기 (예: 게시글 목록의 특정 요소)
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#stock-content > div > div > section > section > ul > div > div'))
                )
            except Exception as e:
                print("로딩 대기 중 오류 발생:", e)
                break  # 로딩 중 오류가 발생하면 루프 종료

        # 데이터프레임 생성
        post_infos = {
            '닉네임': post_nickname,
            '날짜': post_date,
            '내용': all_contents,
            '종목': code
        }

        post_df = pd.DataFrame(post_infos)
        all_post_data.append(post_df)
        driver.quit()
        
    final_df = pd.concat(all_post_data, ignore_index=True)   
    final_df.to_csv(f'practice_Toss_{market}_{today_str}.csv', index=False, encoding='utf-8-sig') 
    return final_df

(시가총액 상위 5종목) - 8분30초( 약 1700건)

In [34]:
result = crawling_community('ETF', start_date='2024-12-12')
result3 = crawling_community('KOSPI', start_date='2024-12-12')

Processing 459580 crawling...
https://tossinvest.com/stocks/A459580/community
Processing 360750 crawling...
https://tossinvest.com/stocks/A360750/community
Processing 357870 crawling...
https://tossinvest.com/stocks/A357870/community
Processing 069500 crawling...
https://tossinvest.com/stocks/A069500/community
Processing 133690 crawling...
https://tossinvest.com/stocks/A133690/community
Processing 488770 crawling...
https://tossinvest.com/stocks/A488770/community
Processing 423160 crawling...
https://tossinvest.com/stocks/A423160/community
Processing 379800 crawling...
https://tossinvest.com/stocks/A379800/community
Processing 449170 crawling...
https://tossinvest.com/stocks/A449170/community
Processing 381170 crawling...
https://tossinvest.com/stocks/A381170/community
Processing 273130 crawling...
https://tossinvest.com/stocks/A273130/community
Processing 381180 crawling...
https://tossinvest.com/stocks/A381180/community
Processing 278530 crawling...
https://tossinvest.com/stocks/A278

IndexError: single positional indexer is out-of-bounds

In [40]:
result3 = crawling_community('KOSPI', start_date='2024-12-12')

Processing 175330 crawling...
https://tossinvest.com/stocks/A175330/community
Processing 006260 crawling...
https://tossinvest.com/stocks/A006260/community
Processing 009830 crawling...
https://tossinvest.com/stocks/A009830/community
Processing 039490 crawling...
https://tossinvest.com/stocks/A039490/community
Processing 022100 crawling...
https://tossinvest.com/stocks/A022100/community
Processing 066970 crawling...
https://tossinvest.com/stocks/A066970/community
Processing 004020 crawling...
https://tossinvest.com/stocks/A004020/community
Processing 000720 crawling...
https://tossinvest.com/stocks/A000720/community
Processing 001040 crawling...
https://tossinvest.com/stocks/A001040/community
Processing 018880 crawling...
https://tossinvest.com/stocks/A018880/community
Processing 011780 crawling...
https://tossinvest.com/stocks/A011780/community
Processing 011170 crawling...
https://tossinvest.com/stocks/A011170/community
Processing 081660 crawling...
https://tossinvest.com/stocks/A081

In [ ]:
# 80분 시가총액 상위 10종목 커뮤니티티18751건
# 600분 시가총액 상위 100종목 커뮤니티 15만건 예상

In [ ]:
df.shape()

코스피 상위 100종목 : 3.7만건 3시간
코스닥 상위 1000종목 : 2.46만건 2사건

In [21]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24657 entries, 0 to 24656
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   닉네임     24657 non-null  object
 1   날짜      24657 non-null  object
 2   내용      24657 non-null  object
 3   종목      24657 non-null  object
dtypes: object(4)
memory usage: 770.7+ KB


In [88]:
result.to_csv(f'practice_1.csv', index=False, encoding='utf-8-sig')

In [59]:
post_df

,닉네임,날짜,내용
0,구부랑길고양이,2025-01-10T11:32:47+09:00,진짜 앞자리 6을 못보네
1,부자71,2025-01-10T11:31:00+09:00,세상 상황을 아는것도 주식과 연결되듯이\n중국은 내정간섭중입니다.\n정치인과 결탁되...
2,슈랭슈랭,2025-01-10T11:30:20+09:00,울 삼전이는 시간이 약이네요.스트레스 지지~\n어제 뉴스에 댓글에 댓글보고 혼자 빵...
3,공동우승,2025-01-10T11:30:16+09:00,4만 가즈아~~~
4,쓉거지키우기,2025-01-10T11:30:16+09:00,얘네 뭐하냐
...,...,...,...
2956,Youngest,2025-01-06T07:47:52+09:00,성과급 자랑하지말자 소액 주주들 피똥싸는 중이다
2957,단타는몸에좋다,2025-01-06T07:14:48+09:00,단체로 정치에 미쳐서 삼성 이야기는 안하고 정치얘기밖에 없네
2958,Youngest,2025-01-06T05:49:34+09:00,나 또 왔다 ㅋㅋ
2959,자민당,2025-01-06T02:00:26+09:00,쩝.


In [32]:
print(len(post_df))

747


In [33]:
# post_df = post_df.drop_duplicates()

In [61]:
post_df.to_csv(f'practice_Toss_{code}.csv', index=False, encoding='utf-8-sig')

In [6]:
df = pd.read_csv('practice_Toss_2025-01-11.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18976 entries, 0 to 18975
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   닉네임     18976 non-null  object
 1   날짜      18976 non-null  object
 2   내용      18976 non-null  object
 3   종목      18976 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 593.1+ KB


In [22]:
kospi = pd.read_csv('practice_Toss_2025-01-11.csv')
kosdaq = pd.read_csv('practice_Toss_2025-01-12.csv')

In [24]:
combine = pd.concat([kospi,kosdaq])

In [29]:
combine.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61665 entries, 0 to 24656
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   닉네임     61665 non-null  object
 1   날짜      61665 non-null  object
 2   내용      61665 non-null  object
 3   종목      61665 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 2.4+ MB


In [28]:
combine.to_csv('toss_crawling_1m_top_100_market.csv', index=False, encoding='utf-8-sig')

In [ ]:
kospi_code_list = code_list_by_market('KOSPI')

In [37]:
df = fdr.StockListing('KOSPI').sort_values(by='Marcap', ascending=False).head(100)


In [38]:
df = df.iloc[100:400]

In [7]:
combine = pd.read_csv('toss_crawling_1m_top_100_market.csv')

In [8]:
kospi_under = pd.read_csv('practice_Toss_KOSPI_2025-01-12.csv')
etfs = pd.read_csv('practice_Toss_ETF_2025-01-12.csv')

In [9]:
final = pd.concat([combine,kospi_under,etfs])

In [10]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81072 entries, 0 to 2233
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   닉네임     81072 non-null  object
 1   날짜      81072 non-null  object
 2   내용      81072 non-null  object
 3   종목      81072 non-null  object
dtypes: object(4)
memory usage: 3.1+ MB


In [11]:
final.to_csv('toss_crawling_1m.csv',index=False, encoding='utf-8-sig')